 02_model_comparison.ipynb
 -------------------------
 Purpose:
 - This notebook is focused on MODEL SELECTION, not raw data exploration.
 - It compares multiple regression models on the same dataset and decides
   which model is best, based on metrics and cross-validation.

 What this notebook does:
 1. Load the cleaned features and target:
    - Uses build_features(df) to get X (features) and y (target).
    - This ensures we use the same feature logic as the rest of the project.
 2. Create a train/test split:
    - Uses train_test_split_xy(X, y) from the shared features module.
    - Train set: used to fit the models.
    - Test set: used to evaluate generalisation on unseen data.
 3. Apply a log-transform to the target for modelling:
    - We model log(price) instead of price directly to:
        - Reduce skew in the target.
        - Stabilise variance (reduce heteroscedasticity).
        - Improve linear model assumptions.
    - Models are fit on log(y), but evaluation is done back in original price units.
 4. Define and compare multiple models:
    - Linear Regression (plain OLS).
    - Ridge Regression (L2 regularisation).
    - Lasso Regression (L1 regularisation).
    - Each model is wrapped in a Pipeline with:
        - "preprocess": build_feature_pipeline() (handles numeric + categorical).
        - "model": the chosen regressor.
 5. Evaluate models on the test set:
    - For each model:
        - Fit on X_train, log(y_train).
        - Predict on X_test, get log predictions.
        - Exponentiate predictions back to price scale.
        - Compute regression metrics (MAE, MSE, RMSE, R2) in price units.
    - Store results in a DataFrame for side-by-side comparison.
 6. Evaluate models with cross-validation:
    - Use cross_val_score with a log-target to compute CV RMSE.
    - This gives a more stable comparison than a single train/test split.
    - Helps avoid overfitting to one particular split.
 7. Select a "best" model:
    - Based on RMSE, MAE, R2, and CV RMSE.
    - In this project, Ridge typically wins because:
        - The data has very strong multicollinearity between numeric features.
        - Ridge shrinks coefficients and stabilises the model.
 8. Do basic diagnostics on the chosen model:
    - For the selected model (e.g. Ridge):
        - Predict on the test set (in price units).
        - Plot residuals vs predicted price.
        - Plot residual histogram.
        - Plot Q-Q plot of residuals.
    - These quick checks confirm that the chosen model behaves sensibly.

 What this notebook does NOT do:
 - It does NOT re-do general EDA (no df.info, df.describe, general histograms).
 - It does NOT finalise interpretation or production deployment.

 How this notebook fits into the overall workflow:
 - 01_exploration.ipynb tells us what the data looks like.
 - 02_model_comparison.ipynb tells us which type of model works best
   (Linear vs Ridge vs Lasso) and with what settings (e.g. alpha).
 - Once a best model is chosen (e.g. Ridge on log(price)), that choice is
   reflected in the project code (e.g. MODEL_REGISTRY in train.py).
 - Detailed diagnostics and interpretability for the final chosen model
   are handled in 03_model_diagnostics.ipynb.


In [1]:
import sys
import os

# Make src/ importable
project_root = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(project_root, "src"))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

from spend_model.data_load import load_raw_data
from spend_model.features import (
    NUMERIC_FEATURES,
    CATEGORICAL_FEATURES,
    TARGET_COL,
    build_features,
    build_feature_pipeline,
    train_test_split_xy,
    engineer_features,
)
from spend_model.evaluation import regression_metrics

In [2]:
df_raw = load_raw_data()
df = engineer_features(df_raw)

X, y = build_features(df)

print("Shape:", X.shape)
display(X.head())
print("\nTarget preview:")
display(y.head())

Shape: (203, 29)


,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,aspiration,doornumber,carbody,drivewheel,enginelocation,enginetype,cylindernumber,fuelsystem,car_brand,car_model
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,std,two,convertible,rwd,front,dohc,four,mpfi,alfa-romero,giulia
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,std,two,convertible,rwd,front,dohc,four,mpfi,alfa-romero,stelvio
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,std,two,hatchback,rwd,front,ohcv,six,mpfi,alfa-romero,quadrifoglio
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,std,four,sedan,fwd,front,ohc,four,mpfi,audi,100
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,std,four,sedan,4wd,front,ohc,five,mpfi,audi,100ls



Target preview:


0    13495.0
1    16500.0
2    16500.0
3    13950.0
4    17450.0
Name: price, dtype: float64

In [ ]:
print(df.shape)

In [3]:
X_train, X_test, y_train, y_test = train_test_split_xy(X, y)

print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)

Train shape: (162, 29)
Test shape : (41, 29)


In [ ]:
def evaluate_model(pipeline: Pipeline, X_train, X_test, y_train, y_test):
    """
    Fit model on log(target) and evaluate in original price units.
    """
    y_train_log = np.log(y_train)

    # Fit model on log(price)
    pipeline.fit(X_train, y_train_log)

    # Predict log(price) then invert to price
    y_pred_log = pipeline.predict(X_test)
    y_pred = np.exp(y_pred_log)

    # Metrics in original price scale
    return regression_metrics(y_test, y_pred), y_pred


def cv_rmse(pipeline: Pipeline, X, y, folds: int = 5) -> float:
    """
    Cross-validated RMSE in log-price space.
    """
    y_log = np.log(y)
    scores = cross_val_score(
        pipeline,
        X,
        y_log,
        scoring="neg_root_mean_squared_error",
        cv=folds,
    )
    return -scores.mean()


In [6]:
pipelines = {
    "Linear": Pipeline([
        ("preprocess", build_feature_pipeline()),
        ("model", LinearRegression()),
    ]),
    "Ridge": Pipeline([
        ("preprocess", build_feature_pipeline()),
        ("model", Ridge(alpha=10.0)),
    ]),
    "Lasso": Pipeline([
        ("preprocess", build_feature_pipeline()),
        ("model", Lasso(alpha=0.001)),
    ]),
    "RandomForest": Pipeline([
        ("preprocess", build_feature_pipeline()),
        ("model", RandomForestRegressor(
            n_estimators=500,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            n_jobs=-1,
            random_state=42,
        )),
    ]),
}

In [ ]:
results = {}
y_pred_store = {}

for name, pipe in pipelines.items():
    print(f"Training {name}...")
    metrics, y_pred = evaluate_model(pipe, X_train, X_test, y_train, y_test)
    results[name] = metrics
    y_pred_store[name] = y_pred

results_df = pd.DataFrame(results).T
results_df

Training Linear...
Training Ridge...
Training Lasso...
Training RandomForest...
{'Linear': {'MAE': 3181.5998157506747, 'MSE': 28129170.774339188, 'RMSE': np.float64(5303.694068697702), 'R2': 0.6314208605699949}, 'Ridge': {'MAE': 2382.345266445799, 'MSE': 12755619.040987263, 'RMSE': np.float64(3571.5009507190757), 'R2': 0.8328619379952369}, 'Lasso': {'MAE': 2119.6012696119474, 'MSE': 11393574.687443493, 'RMSE': np.float64(3375.436962445528), 'R2': 0.8507089317698501}, 'RandomForest': {'MAE': 1557.8980635500345, 'MSE': 5150630.219335017, 'RMSE': np.float64(2269.49999324411), 'R2': 0.9325108134543196}}


,MAE,MSE,RMSE,R2
Linear,3181.599816,2.812917e+07,5303.694069,0.631421
Ridge,2382.345266,1.275562e+07,3571.500951,0.832862
Lasso,2119.601270,1.139357e+07,3375.436962,0.850709
RandomForest,1557.898064,5.150630e+06,2269.499993,0.932511


In [ ]:
cv_results = {}

for name, pipe in pipelines.items():
    print(f"CV for {name}...")
    cv_results[name] = cv_rmse(pipe, X, y, folds=5)

pd.DataFrame(cv_results, index=["CV_RMSE"]).T

In [ ]:
best_name = "RandomForest"
best_pipeline = pipelines[best_name]

# Refit best model on full training data
y_train_log = np.log(y_train)
best_pipeline.fit(X_train, y_train_log)

# Predictions on test set
y_pred_log_best = best_pipeline.predict(X_test)
y_pred_best = np.exp(y_pred_log_best)
residuals = y_test - y_pred_best

print(f"Best model: {best_name}")
regression_metrics(y_test, y_pred_best)

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(y_pred_best, residuals, alpha=0.6)
plt.axhline(0, linestyle="--", color="red")
plt.xlabel("Predicted Price")
plt.ylabel("Residual (Actual - Predicted)")
plt.title(f"Residuals vs Predicted ({best_name})")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(residuals, kde=True)
plt.title(f"Residual Distribution ({best_name})")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
sm.qqplot(residuals, line="45")
plt.title(f"Q-Q Plot of Residuals ({best_name})")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_best, alpha=0.5)

plt.xlabel("Actual " + TARGET_COL)
plt.ylabel("Predicted " + TARGET_COL)
plt.title("Actual vs Predicted")
plt.show()